<a href="https://colab.research.google.com/github/ykitaguchi77/Laboratory_course/blob/master/4.%20%E7%94%BB%E5%83%8F%E3%81%AE%E3%82%B9%E3%82%AF%E3%83%AC%E3%82%A4%E3%83%94%E3%83%B3%E3%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**画像のスクレイピング**



#**iCrawlerを使う方法**

https://atmarkit.itmedia.co.jp/ait/articles/2010/28/news018.html

公式： https://icrawler.readthedocs.io/en/latest/builtin.html

In [4]:
!pip install icrawler
from icrawler.builtin import BingImageCrawler
import os

# List of keywords
keywords = ["cat", "dog", "bird"]
max_num = 10

for keyword in keywords:
    output_dir = f"/content/{keyword}"

    # Create the directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    crawler = BingImageCrawler(storage={"root_dir": output_dir})
    crawler.crawl(keyword=keyword, max_num=max_num)

ERROR:downloader:Exception caught when downloading file https://i1.wp.com/www.ucityinbloom.org/wp-content/uploads/2015/02/american-goldfinch-male.jpg, error: HTTPSConnectionPool(host='i1.wp.com', port=443): Read timed out. (read timeout=5), remaining retry times: 2


##**Requestsを用いる方法**

In [9]:
import requests
from bs4 import BeautifulSoup
import os

def download_images(query, number_of_images):
    url = f"https://www.google.com/search?q={query}&sca_esv=596760372&tbm=isch&sxsrf=ACQVn08jqA_YQTJvXzWkTPH7ZTUxMvP7Tw:1704771354888&source=lnms&sa=X&ved=2ahUKEwiV1p-YsM-DAxVqi68BHcgqDX4Q_AUoAXoECAEQAw&biw=1536&bih=754&dpr=2"

    # Send a request and get the HTML content
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    html = response.text

    # Parse the HTML using BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')
    images = soup.find_all('img')

    # Create a directory to save images
    os.makedirs('images', exist_ok=True)

    # Download the first 10 images
    for i, img in enumerate(images[:number_of_images]):
        img_url = img['src']

        try:
            img_data = requests.get(img_url).content
            with open(f'images/apple_{i+1}.jpg', 'wb') as file:
                file.write(img_data)
            print(f'Downloaded image {i+1}')
        except:
            print("download error")
            pass

download_images('りんご', 10)


download error
Downloaded image 2
Downloaded image 3
Downloaded image 4
Downloaded image 5
Downloaded image 6
Downloaded image 7
Downloaded image 8
Downloaded image 9
Downloaded image 10


In [15]:
import requests
from bs4 import BeautifulSoup
import os
import re
import json

def download_original_images(query, number_of_images):
    url = f"https://www.google.com/search?q={query}&tbm=isch"

    # Send a request and get the HTML content
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    html = response.text


    # Parse the HTML using BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Extract the JavaScript where the image data is located
    script = soup.find("script", text=re.compile("AF_initDataCallback"))

    data = re.search(r"AF_initDataCallback\((.*?)\);", script.string).group(1)

    # Extract the required data
    data_json = json.loads(data)
    image_data = data_json['data'][31][0][12][2]

    # Create a directory to save images
    os.makedirs('images', exist_ok=True)

    # Download the specified number of images
    for i, img in enumerate(image_data[:number_of_images]):
        img_url = img[1][3][0]

        try:
            img_data = requests.get(img_url, headers=headers).content
            with open(f'images/image_{i+1}.jpg', 'wb') as file:
                file.write(img_data)
            print(f'Downloaded image {i+1}')
        except Exception as e:
            print(f"Download error for image {i+1}: {e}")

download_original_images('りんご', 10)


None


<ipython-input-15-bc1a9fac33d2>:20: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  script = soup.find("script", text=re.compile("AF_initDataCallback"))


AttributeError: 'NoneType' object has no attribute 'string'

In [16]:
import requests
import re
import json
from bs4 import BeautifulSoup

def scrape_google_images(query):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"
    }

    params = {
        "q": query,
        "tbm": "isch",
        "hl": "en",
        "gl": "us"
    }

    html = requests.get("https://google.com/search", params=params, headers=headers, timeout=30)
    soup = BeautifulSoup(html.text, "lxml")

    # Extracting inline JSON from script tags
    all_script_tags = soup.select("script")
    matched_images_data = "".join(re.findall(r"AF_initDataCallback\(([^<]+)\);", str(all_script_tags)))

    # Parsing JSON data
    matched_images_data_fix = json.dumps(matched_images_data)
    matched_images_data_json = json.loads(matched_images_data_fix)

    # Extracting image data
    matched_google_image_data = re.findall(r'\"b-GRID_STATE0\"(.*)sideChannel:\s?{}}', matched_images_data_json)

    # Extracting thumbnails
    matched_google_images_thumbnails = ", ".join(
        re.findall(r'\[\"(https\:\/\/encrypted-tbn0\.gstatic\.com\/images\?.*?)\",\d+,\d+\]',
                   str(matched_google_image_data))).split(", ")
    thumbnails = [bytes(bytes(thumbnail, "ascii").decode("unicode-escape"), "ascii").decode("unicode-escape") for thumbnail in matched_google_images_thumbnails]

    # Extracting full resolution images
    removed_matched_google_images_thumbnails = re.sub(
            r'\[\"(https\:\/\/encrypted-tbn0\.gstatic\.com\/images\?.*?)\",\d+,\d+\]', "", str(matched_google_image_data))
    matched_google_full_resolution_images = re.findall(r"(?:'|,),\[\"(https:|http.*?)\",\d+,\d+\]", removed_matched_google_images_thumbnails)
    full_res_images = [bytes(bytes(img, "ascii").decode("unicode-escape"), "ascii").decode("unicode-escape") for img in matched_google_full_resolution_images]

    return full_res_images

# Using the function
images = scrape_google_images("tower")
print(images)


['https://upload.wikimedia.org/wikipedia/commons/3/3e/Tokyo_Sky_Tree_2012.JPG', 'https://upload.wikimedia.org/wikipedia/commons/thumb/8/85/Tour_Eiffel_Wikimedia_Commons_%28cropped%29.jpg/640px-Tour_Eiffel_Wikimedia_Commons_%28cropped%29.jpg', 'https://cdn.britannica.com/54/75854-050-E27E66C0/Eiffel-Tower-Paris.jpg', 'https://www.ubm-development.com/magazin/wp-content/uploads/2021/12/S_TernaryTower_1.jpg', 'https://upload.wikimedia.org/wikipedia/commons/thumb/0/02/Willis_Tower_From_Lake.jpg/1200px-Willis_Tower_From_Lake.jpg', 'https://cdn.britannica.com/51/94351-050-86B70FE1/Leaning-Tower-of-Pisa-Italy.jpg', 'https://www.great-towers.com/themes/worldtower/assets/images/about/1.jpg', 'https://skyscraper.org/supertall/wp-content/uploads/sites/2/2021/11/BJ0618-02-scaled.jpg', 'https://upload.wikimedia.org/wikipedia/commons/thumb/f/ff/Canton_Tower_20220626_%28cropped_2%29.jpg/640px-Canton_Tower_20220626_%28cropped_2%29.jpg', 'https://i.natgeofe.com/k/6d4021bf-832e-49f6-b898-27b7fcd7cbf7/eif

##**AZURE Bing Searchを用いる方法**

準備：
- Microsoft AZUREに登録

    https://learn.microsoft.com/ja-jp/azure/cognitive-services/bing-web-search/

- 左のタブ → リソースの作成 → Bing Search v7を取得

- ダッシュボード → キーとエンドポイントからキーを取得する

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

with open("/content/drive/MyDrive/Deep_learning/api.txt") as file:
    #text = file.read()
    i=1
    key = []
    while True:
        include_break_line = file.readline() #改行が含まれた行
        line = include_break_line.rstrip() #改行を取り除く
        if line: #keyの読み込み
            #print(f'{i}行目：{line}')
            key.append(line)
            i += 1
        else:
            break

bing_api_key = key[13]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import csv

from requests import exceptions
import argparse
import requests
import cv2
import os

API_KEY = f"{bing_api_key}"
MAX_SIZE = 10
GROUP_SIZE = 5

# 取得したエンドポイントURL
URL = "https://api.bing.microsoft.com/v7.0/images/search"
OUTPUT = '/content/save_dir'

if not os.path.isdir(OUTPUT):
    os.mkdir(OUTPUT)

EXCEPTIONS = set([IOError, FileNotFoundError,
    exceptions.RequestException, exceptions.HTTPError,
    exceptions.ConnectionError, exceptions.Timeout])

search_terms = ["forest", "river", "house"]

# set the output csv file name
csv_file = "url_list.csv"

# create the csv file and write the headers
with open(csv_file, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['Search term', 'Image URL'])

# loop over each search term and download images
for term in search_terms:
    print(f"[INFO] searching Bing API for '{term}'")

    # create the directory to save the images for the current search term
    output_dir = os.path.join(OUTPUT, term)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    headers = {"Ocp-Apim-Subscription-Key": API_KEY}
    params = {"q": term, "offset": 0, "count": GROUP_SIZE, "imageType": "Photo", "color": "ColorOnly"}

    # make the search
    search = requests.get(URL, headers=headers, params=params)
    search.raise_for_status()

    # grab the results from the search, including the total number of
    # estimated results returned by the Bing API
    results = search.json()
    est_num_results = min(results["totalEstimatedMatches"], MAX_RESULTS)
    print(f"[INFO] {est_num_results} total results for '{term}'")

    # initialize the total number of images downloaded thus far
    total = 0

    # loop over the estimated number of results in `GROUP_SIZE` groups
    for offset in range(0, est_num_results, GROUP_SIZE):
        # update the search parameters using the current offset, then
        # make the request to fetch the results
        params["offset"] = offset
        search = requests.get(URL, headers=headers, params=params)
        search.raise_for_status()
        results = search.json()

        # loop over the results
        for v in results["value"]:
            # try to download the image
            try:
                # make a request to download the image
                print("[INFO] fetching: {}".format(v["contentUrl"]))
                r = requests.get(v["contentUrl"], timeout=30)

                # build the path to the output image
                ext = v["contentUrl"][v["contentUrl"].rfind("."):]
                filename = f"{term}_{str(total).zfill(3)}{ext}"
                output_path = os.path.join(output_dir, filename)

                # write the image to disk
                with open(output_path, "wb") as f:
                    f.write(r.content)

                # write the URL to the csv file
                with open(csv_file, 'a', newline='') as f:
                    writer = csv.writer(f)
                    writer.writerow([term, v["contentUrl"]])

            # catch any errors that would not unable us to download the
            # image
            except Exception as e:
                print(f"[INFO] skipping: {v['contentUrl']}")

            # if we have reached the maximum number of images, break out
            # of the loop
            total += 1
            print(f"{total} images downloaded!")
            if total >= MAX_SIZE:
                break

        # if we have reached the maximum number of images, break out of
        # the loop
        if total >= MAX_SIZE:
            break


##**Chromedriverを用いる方法**

In [ ]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium==4.1.0 #新しいバージョンだとエラーが出るので旧バージョンにする

In [ ]:
# これだとサムネイルしか取得できない

import requests
from bs4 import BeautifulSoup
import os

# Search query
search_query = "flowers"

# Number of images to download
num_images = 10

# Create a new folder for the images
if not os.path.exists(search_query):
    os.makedirs(search_query)

# URL to search Google Images
url = f"https://www.google.com/search?q={search_query}&tbm=isch"

# Send GET request
response = requests.get(url)

# Parse the HTML using Beautiful Soup
soup = BeautifulSoup(response.text, 'html.parser')

# Find all image tags
images = soup.find_all('img')

# Iterate through the images and download them
for i, img in enumerate(images[:num_images]):
    url = img['src']
    print(i)
    try:
        response = requests.get(url)
        open(f"{search_query}/{search_query}_{i}.jpg", "wb").write(response.content)
    except:
        print("download error")

In [ ]:
!apt-get update
!curl -O https://chromedriver.storage.googleapis.com/110.0.5481.77/chromedriver_linux64.zip #Chromeのバージョンに合ったchromedriverのアドレスを設定
!unzip chromedriver_linux64.zip
!chmod +x chromedriver
!mv chromedriver /usr/local/bin/
!pip install selenium

from selenium import webdriver

# Chromeドライバーの設定
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-gpu')
options.add_argument('--disable-browser-side-navigation')

# Googleで検索する
search_query = 'flowers'
url = f"https://www.google.com/search?q={search_query}&tbm=isch"
browser = webdriver.Chrome('chromedriver',options=options)
browser.get(url)


import os
from bs4 import BeautifulSoup
import base64
import requests
from io import BytesIO
from PIL import Image


# 画像のURLを取得する
soup = BeautifulSoup(browser.page_source, 'html.parser')
img_tags = soup.find_all('img', class_='rg_i')


urls = []
for img in img_tags:
    try:
        urls.append(img["src"])
    except:
        pass


# 画像をダウンロードする
if not os.path.exists(search_query):
    os.makedirs(search_query)

num_images = 10

counter = 0
for i in range(num_images):
    print(urls[i])
    image_data = base64.b64decode(urls[i].split(',')[1])

    # バイナリデータをBytesIOオブジェクトに書き込む
    image_stream = BytesIO(image_data)

    # PILで画像オブジェクトを作成する
    image = Image.open(image_stream)
    image_format = image.format

    # 画像のネーミング
    num= "{:04d}".format(i)
    file_name = f"{search_query}_{num}"
    new_image_path = f"{search_query}/{file_name}.{image_format}"


    # Save image to file
    image.save(new_image_path)
